In [782]:
import numpy as np
from sympy import symbols, And, Not, simplify, solve, reduce_inequalities
%run constants.ipynb
%run li_handler.ipynb

In [2]:
def no_loser(a,b,c):
    return [simplify(i) for i in [a>0, b>0, c>0]]

In [3]:
def parentStates(a,b,c, r=V):
    return [s for s in 
            [(2*a,b-a,c), (2*a,b,c-a),
            (a-b,2*b,c), (a,2*b,c-b),
            (a-c,b,2*c), (a,b-c,2*c)] 
            if feasible(*no_loser(*s), r=r)]

In [4]:
@memoized
def coef(n):
    return 1/(6**n)

In [5]:
def concat(ls):
    ''' Flatten 2D list. '''
    return [j for l in ls for j in l]

In [8]:
import csv
from itertools import islice

def read(filename, min_n=1, max_n=5):
    hs = {}
    with open(filename, 'r') as fp:
        reader = islice(csv.reader(fp), min_n-1, max_n)
        for row in reader:
            n, c, rs = row
            # parse entries
            hs[int(n)] = (int(c), sympify(rs))
    return hs

In [701]:
def h_mult(n,a,b,c):
    return non_redundant(*no_loser(a,b,c), r=V)

def mult_inds(inds, prereqs, r=V):
    '''
    prereqs * sum(inds), keeping only non-redundant indicators in `prereqs` given restriction `r`.
    '''
    return [i + [p for p in non_redundant(*prereqs, r=V+i)] for i in inds]

@memoized
def _h(n,a,b,c):
    ''' Returns the additive indicators of h_n(a,b,c), as a 2D list. '''
    # constant coef = (1/6)^n
    # so we only keep track of the indicators to be summed up
    if n == 1:
        return mult_inds([[simplify(a<=b)], [simplify(a<=c)]], h_mult(1,a,b,c))
    # list of inequalities representing the region Rn
    return mult_inds(concat([_h(n-1, *s) for s in parentStates(a,b,c)]), 
                     h_mult(n,a,b,c))

@memoized
def h(n,a,b,c, regenerate):
    if regenerate:
        return reduce(_h(n,a,b,c), r=V)
    else:
        if (a,b,c) == (y,x,z):
            return read(H_YXZ_CACHE, max_n=n)[n]
        else:
            return read(H_XYZ_CACHE, max_n=n)[n]

# Use xyz

In [702]:
import gurobipy as gp
from gurobipy import GRB
from itertools import repeat

In [842]:
n = 4
H_XYZ = read(H_XYZ_CACHE, min_n=1, max_n=n)
H_YXZ = read(H_YXZ_CACHE, min_n=1, max_n=n)
num_pos_per_n = [len(H_XYZ[i][1]) for i in range(1,n+1)]
num_neg_per_n = [len(H_YXZ[i][1]) for i in range(1,n+1)]
# num_pos_per_n = [len(concat(H_XYZ[i][1])) for i in range(1,n+1)]
# num_neg_per_n = [len(concat(H_YXZ[i][1])) for i in range(1,n+1)]
num_pos_per_n

[0, 5, 31, 163]

In [843]:
num_pos = sum(num_comp_pos_per_n)  # number of positive compound indicators
num_neg = sum(num_comp_neg_per_n)
# num_pos = sum(num_pos_per_n)  # flattened indicators
# num_neg = sum(num_neg_per_n)
num_pos

36

In [844]:
pos_coefs = concat([repeat(j, k) for j,k in zip([coef(i) for i in range(1,n+1)], num_pos_per_n)])
neg_coefs = concat([repeat(j, k) for j,k in zip([-coef(i) for i in range(1,n+1)], num_neg_per_n)])
len(pos_coefs)

199

In [845]:
const = sum([coef(i)*(H_XYZ[i][0] - H_YXZ[i][0]) for i in range(1,n+1)])

In [846]:
all_pos_comp_inds = concat([H_XYZ[i][1] for i in range(1,n+1)])
all_neg_comp_inds = concat([H_YXZ[i][1] for i in range(1,n+1)])
all_pos_comp_inds

[[3*x <= y],
 [2*x <= z],
 [2*x <= y],
 [3*x <= z],
 [x <= -y + z],
 [7*x <= y],
 [4*x <= z, 3*x < y],
 [5*x <= y],
 [6*x <= z],
 [5*x <= 3*y, y < 3*x],
 [3*x <= y + z, y < 3*x],
 [2*x <= y],
 [x <= -y + z],
 [3*x <= y + z, z < 2*x],
 [z < 2*x],
 [6*x <= y],
 [5*x <= z, 2*x < y],
 [4*x <= y],
 [7*x <= z],
 [y < 2*x],
 [3*x <= y + z, y < 2*x],
 [x < -y + z],
 [3*x <= -y + z],
 [3*x <= y + z, z < 3*x],
 [5*x <= 3*z, z < 3*x],
 [2*x <= z, -y + z < x],
 [3*x <= 2*y],
 [2*x <= -y + z],
 [x < -y + z],
 [3*x <= -y + z],
 [3*y < z],
 [x <= -3*y + z],
 [-y + z < x],
 [x <= -3*y + 3*z, -y + z < x],
 [x <= 3*y - z],
 [x <= -2*y + 2*z, z < 3*y],
 [15*x <= y],
 [8*x <= z, 7*x < y],
 [11*x <= y],
 [12*x <= z, 3*x < y],
 [13*x <= 3*y, y < 7*x],
 [7*x <= y + z, y < 7*x, 3*x < y],
 [5*x <= y],
 [x <= -y + z, 3*x < y],
 [7*x <= y + z, z < 4*x, 3*x < y],
 [4*x <= 3*z, z < 4*x, 3*x < y],
 [13*x <= y],
 [10*x <= z, 5*x < y],
 [9*x <= y],
 [14*x <= z],
 [7*x <= 3*y, y < 5*x],
 [7*x <= y + z, y < 5*x],
 [3*x

In [847]:
model = gp.Model()
# model.Params.LazyConstraints = 1
model.Params.OutputFlag = 1
model.Params.LogToConsole = 1
# model.Params.MIPFocus = 1
# model.Params.Heuristics = 1
# model.Params.CliqueCuts = 1
# model.Params.FeasibilityTol = 1e-9
# model.Params.IntFeasTol = 1e-9
# model.Params.MarkowitzTol = 1e-4
# model.Params.NodeLimit = 10000
gx = model.addVar(lb=0, vtype='I', name='x')
gy = model.addVar(lb=1, vtype='I', name='y')
gz = model.addVar(lb=2, vtype='I', name='z')

In [848]:
from sympy import symbols, Add
from gurobipy import Model, LinExpr

# Define SymPy variables
x,y,z = symbols('x y z')

# Mapping between SymPy and Gurobi variables
SP_TO_GP = {x: gx, y: gy, z: gz}

def to_gp_linexpr(sympy_expr, var_map=SP_TO_GP):
    gurobi_expr = LinExpr()

    if isinstance(sympy_expr, Add):
        for term in sympy_expr.args:
            gurobi_expr.add(sympy_to_gurobi_linexpr(term, var_map))
    elif sympy_expr.is_Mul:
        coeff = 1
        var = None
        for factor in sympy_expr.args:
            if factor.is_number:
                coeff *= float(factor)
            elif factor in var_map:
                var = var_map[factor]
        if var is not None:
            gurobi_expr.addTerms(coeff, var)
        else:
            gurobi_expr.addConstant(coeff)
    elif sympy_expr in var_map:
        gurobi_expr.addTerms(1.0, var_map[sympy_expr])
    elif sympy_expr.is_number:
        gurobi_expr.addConstant(float(sympy_expr))

    return gurobi_expr

In [849]:
M = 50
# to non-stricten strict inequalities
# expressions are in terms of x,y,z, which are all ints
EPS = 1  
def add_ind_constr(ind, var):
    r = unify_ineq(ind)  # to the form <= 0 or > 0
    lhs = to_gp_linexpr(r.lhs)
    if r.rel_op == '>':
        model.addConstr(lhs >= EPS - M * (1 - var))
        model.addConstr(lhs <= M * var - EPS)
    elif r.rel_op == '<=':
        model.addConstr(lhs <= M * (1 - var))
        model.addConstr(lhs >= EPS - M * var)
    else:
        raise ValueError 

In [850]:
# for each compound indicator group, restrict their sum to be [0,1]
def add_comp_ind(comp_ind):
    ''' Adds variables and constraints for a compound indicator. '''
    I = model.addVar(vtype=GRB.BINARY, name='main_ind')
    component_vars = model.addVars(len(comp_ind), vtype=GRB.BINARY, name='component_ind').values()
    
    for c in component_vars:
        model.addConstr(I <= c)
    model.addConstr(I >= gp.quicksum(component_vars) - (len(component_vars)-1))
    
    # big-M method to enforce semantics of inequalities
    # if the inequality holds, then the corresponding var is true
    for ineq, c in zip(comp_ind, component_vars):
        add_ind_constr(ineq, c)
    
    return I, list(component_vars)

In [851]:
def add_single_ind(ind):
    I = model.addVar(vtype=GRB.BINARY, name='single_ind')
    add_ind_constr(ind[0], I)
    return (I,)  # no component variables

In [852]:
def add_ind(ind):
    return add_single_ind(ind) if len(ind) == 1 else add_comp_ind(ind)

In [853]:
pos_inds = [add_ind(i) for i in all_pos_comp_inds]
neg_inds = [add_ind(i) for i in all_neg_comp_inds]
pos_inds

[(<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,
  [<gurobi.Var *Awaiting Model Update*>,
   <gurobi.Var *Awaiting Model Update*>]),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,
  [<gurobi.Var *Awaiting Model Update*>,
   <gurobi.Var *Awaiting Model Update*>]),
 (<gurobi.Var *Awaiting Model Update*>,
  [<gurobi.Var *Awaiting Model Update*>,
   <gurobi.Var *Awaiting Model Update*>]),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,
  [<gurobi.Var *Awaiting Model Update*>,
   <gurobi.Var *Awaiting Model Update*>]),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),


In [854]:
obj = gp.quicksum([pos_coefs[i]*pos_inds[i][0] for i in range(num_pos)]) + \
        gp.quicksum([neg_coefs[i]*neg_inds[i][0] for i in range(num_neg)]) + const

In [855]:
model.setObjective(obj, GRB.MINIMIZE)
model.addConstr(obj <= thresh(n))
# constraints for V
model.addConstr(gx <= gy - 1)
model.addConstr(gy <= gz - 1)

<gurobi.Constr *Awaiting Model Update*>

In [856]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

In [837]:
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for v in model.getVars():
        print(f"{v.varName}: {v.x}, {not (not v.x)}")
    print(f"Objective value: {model.objVal}")
else:
    print("No feasible solution found.")

Optimal solution found:
x: 4.0, True
y: 5.0, True
z: 6.0, True
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True
single_ind: 0.0, False
single_ind: 0.0, False
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True
single_ind: 1.0, True
single_ind: 0.0, False
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 1.0, True
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True
single_ind: 0.0, False
single_ind: 0.0, False
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True


In [840]:
# 3: 4,5,6
# 4: 0.5, 1.5, 4.5
def eval_f(n,a,b,c):
    tot = 0.
    vals = {x:a, y:b, z:c}
    for i in range(1,n+1):
        pos_c, pos_rs = H_XYZ[i]
        neg_c, neg_rs = H_YXZ[i]
#         print(sum([sub(r, vals) for r in neg_rs]))
        tot += coef(i) * (pos_c - neg_c + sum([sub(r, vals) for r in pos_rs]) - sum([sub(r, vals) for r in neg_rs]))
    return tot - thresh(n)

In [841]:
eval_f(3,4,5,6)

-0.016666666666666677